## Package management in Python
#### Background information
* The standard package manager in python is pip (Pip Installs Packages)
* The standard central repository for python packages is PyPI (Python Package Index)

### What's wrong with just using one Python installation for everything?
#### What if I want to use different versions of Python for different projects?
Well, you could install another version of python and add it to your path.

Then what happens when you use pip?  You can't really be sure without additional information.  You'd have to know which version of python is first on your path, and maybe specify the whole path to the python you want to use to be sure.

#### What if I have two projects that depend on conflicting versions of the same package?
This is a big one: Python can't handle this on its own.  If ProjectA requires basicpackage 2.0 and ProjectB requires basicpackage 3.0, you're out of luck, they can't both exist on the same python.  

#### How do I document dependencies for other users?
You can look at every file for the import statements. Or, you can fail upwards by repeatedly running the code, see which import fails, install that package, and repeat until the software works.

_What if there were a better way?_

## Enter the Virtual Environment!
#### What is a virtual environment?
* A virtual environment is a way to easily create a fresh python installation for your project.  Each one can have its own packages with whatever versions it needs.


#### Why should I use one?
It is Best Practice™ to have a separate virtual environment for every project you work on because:
* It becomes trivial to guarantee that every place you use python (your terminal, IDE debugger and terminal, etc) is using the same python installation.
* Because your dependencies are isolated, they easy to document and easy to replicate.

#### How does it work?
* A virtual environment creates a new Python installation directory. 
* In the bin/ subdirectory, it has hardlinks to the binaries of another, "base" Python installation directory.
* It points to its *own* `site-packages` or equivalent directories where packages can be installed.
* When we activate a virtual environment, it sets your environment variables to point to the new, hardlinked python.
* When we deactivate it, everything goes back to how it was before.

There are several packages that allow for creating python virtual environments (the default venv is good and easy to use) but my favorite is...
## Conda
* Conda is an open-source python version manager *and* package manager *and* environment manager that runs on Windows, macOS and Linux.
* Conda easily creates, saves, loads and switches between environments on your local computer.
* It was created for Python data scientists (and that's where it is most popular), but it can theoretically package and distribute software for any language (whereas pip and PyPI are only for python packages).
* It has better dependency conflict resolution than pip
    - Pip will install a package that breaks your code (or, worse, silently makes it impossible to replicate your results).
    - Conda will find a way to choose compatible versions or tell you if you've asked for the impossible.
* Despite the above, you can still use pip from within a conda environment, so you're not limited to only Anaconda's packages.
* Some of conda's data science libraries are optimized with the Intel Math Kernel Library for faster training.

## Sounds great, how do I get started?

![Switching from Conda's base environment, to my default environment, to the environment I created to make this presentation](switching-environments.png "Switching Between Conda Environments")

No more worrying about which python installation to use:  Just run `python my_script.py` and the python of your current environment will be used to run it.

## Documentation
![image.png](drake_requirements.jpeg)
* When your code is ready, run `conda env export > environment.yml`.
* Include the environment.yml file in your git repository for future users of your code.
* To create a new environment from an environment.yml file, `conda env create -f envname.yml`

#### I've heard of conda, Anaconda, and miniconda.  What's the difference?
* Conda is the package and environment manager software.
* Anaconda and Miniconda are both distributions.  They both have conda and a python installation that is used as the base.
* Miniconda comes with a minimal python installation and the conda package and environment manager.
* Anaconda is a metapackage of 160 data science packages (numpy, scipy, pandas, etc).
* Installing Anaconda is identical to installing conda + `conda install anaconda`.